### Importing relevant libraries

In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
import re
import csv
import time
import numpy as np
import pandas as pd

In [2]:
base_url = 'https://animeseries.so/'
url_template = 'https://animeseries.so/search/character=special'
url = base_url

### making string of all alphabets for anime list

In [3]:
categ = []
x = 'A'
while x <= 'Z':
    categ.append(x);
    x = chr(ord(x) + 1)
    
categ.insert(0, 'special')
print(categ)

['special', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


### Scraping data to a list of map

In [12]:
Anime = []
anime_dict = {}
main_start = time.time()

for alphabet in categ[0:24]:
    
    print('\rProcessing {}.'.format(alphabet), end = '')
    url = re.sub('special', alphabet, url_template)
    
    url_link = Request(url, headers = {'User-Agent': 'Chrome/113.0.5672.94'})
    url_page = urlopen(url_link).read()
    url_html = bs(url_page, 'html.parser')
    
    containers = url_html.find_all('div', class_='content_episode revent datagrild')
    print("\rProcessing {}..".format(alphabet), end='')
    
    for items in containers[:]:
        animes = items.find_all('div',class_ = "name")
        html_links = items.find_all('a') #getting url of anime
        
        for i,anime in enumerate(animes[:]):
            name=''
            description=''
            season=''
            genre=''
            new_url=''
            status=''
            episodes=''
            status
            initial_air_date=''
            movie=''
            sub=''
            
            
            try:
                title = anime.text#find('div',class_ = "name").text #getting name of anime
#                 print(title)
                html_link = html_links[i].get('href') #getting url of anime
#                 print(main_url)
#                 print(base_url + html_link)

                #Checking if Season in mentioned in title & extracting both name and season seperatly
                #using  regex search function to extract Anime Title and Season seperatly
                if re.search(r"Season [0-9]?",title):
                    name,season = re.split(r"Season [0-9]?",title) #spliting title content based on " Season " text
                elif re.search(r"[0-9][a-z]+? Season",title):
                    name,season = re.split(r"[0-9][a-z]+? Season",title) #spliting title content based on " Season " text
                elif re.search(r"[0-9][a-z][a-z]",title):
                    try:
                        name,season = re.split(r"[0-9]['nd','rd','th']+",title) #spliting title content based on " Season " text
                    except:
                        name = re.split(r"[0-9]['nd','rd','th']+",title)[0] #spliting title content based on " Season " text
                else:
                    name = title #if no " Season " text is found in the Anime title Initilizing name = title

                #initilizing a pattern to search for " Season " in the Anime Title using regex
                pattern = r"^([0-9][a-z]+? Season | Season [0-9][a-z]+?\s )$"
                season = re.search(pattern,title) #using regex search function to search for pattern

                if not season:
                    pattern = r"[0-9]['nd','rd','th']+"
                    season = re.search(pattern,title) #using regex search function to search for pattern

                if season:
                    pass #if season is already found do nothing
                else:
                    season = ["Season 1"] #if season is still not found Initilizing season = ["Season 1"]
                
                #Initilizing " sub " variable as 'Dub' or 'Sub' based on its Apperance in Title
                if 'Dub' in title or 'dub' in title:
                    sub = 'Dub'
                else:
                    sub = 'Sub'
                

                #creating new_url to reroute to anime main page to extract more detatiled information such as:
                #1. description
                #2. status
                #3. initial air date
                #4. genre
                #5. total number of episodes
                new_url = base_url+html_link
                new_url = new_url.replace('watch','anime')
#                 print(main_url)
#                 print(html_link)
                url_link = Request(new_url, headers={'User-Agent': 'Chrome/113.0.5672.94'})
                url_page = urlopen(url_link).read()
                url_html = bs(url_page, 'html.parser')

                #getting div's with class_="main_body"
                containers = url_html.find_all('div',class_="main_body")
                #getting div's with class_="right"
                contents = containers[0].find_all('div',class_='right')


                #extracting description of anime
                description = contents[0].find_all('p')[0].text 
                #extracting status, initial air date, genre & total number of episodes of anime
                temp=[]
                genre=[]
                for i in contents[0].find_all('a')[2:]:
                    temp.append(i.text)

                status,initial_air_date = temp[:2]
                genre = temp[2:]
                genre = ', '.join(genre)

                containers = containers[0].find_all('div',class_='list_episode')
                try:
                    text = containers[0].find('span',class_='name').text
                    text = re.search('Episode [0-9]+',text)
                    episodes = re.search('[0-9]+',text[0])[0]
                    if int(episodes) > 0 and int(episodes) < 2:
                        movie = 'Movie'
                    else:
                        movie = 'Series'
                except:
                    episodes = 'None'
                    movie = 'None'


                print("\rProcessing {}...".format(alphabet),end='')

                #adding all extracted information in Anime dict
                anime_dict['Anime Title'] = name
                anime_dict['Description'] = description
                anime_dict['Current/Latest Season'] = season[0]
                anime_dict['URL'] = new_url
                anime_dict['Status'] = status
                anime_dict['Initial Air Date'] = initial_air_date
                anime_dict['Genre'] = genre
                anime_dict['Episodes Aired'] = episodes
                anime_dict['Series/Movie'] = movie
                anime_dict['Sub/Dub'] = sub

                new_dict = anime_dict.copy()

                Anime.append(new_dict)
                
            except:

                print("\rProcessing {}...".format(alphabet),end='')
                
                try:
                    #adding all extracted information in Anime dict
                    anime_dict['Anime Title'] = name
                    anime_dict['Description'] = description
                    anime_dict['Current/Latest Season'] = season[0]
                    anime_dict['URL'] = new_url
                    anime_dict['Status'] = status
                    anime_dict['Initial Air Date'] = initial_air_date
                    anime_dict['Genre'] = genre
                    anime_dict['Episodes Aired'] = episodes
                    anime_dict['Series/Movie'] = movie
                    anime_dict['Sub/Dub'] = sub
                except:
                    pass

                new_dict = anime_dict.copy()

                Anime.append(new_dict)
                
#                 break
    
    print("\rProcessed {}     ".format(alphabet))
#     break
    
main_end = time.time() - main_start
print('\rWeb Scraping Complete')
print("Total Time Taken: {}s".format(main_end))

Processed special     
Processed A     
Processed B     
Processed C     
Processed D     
Processed E     
Processed F     
Processed G     
Processed H     
Processed I     
Processed J     
Processed K     
Processed L     
Processed M     
Processed N     
Processed O     
Processed P     
Processed Q     
Processed R     
Processed S     
Processed T     
Processed U     
Processed V     
Processed W     
Web Scraping Complete
Total Time Taken: 11335.190744638443s


In [13]:
for alphabet in categ[24:]:
    
    print('\rProcessing {}.'.format(alphabet), end = '')
    url = re.sub('special', alphabet, url_template)
    
    url_link = Request(url, headers = {'User-Agent': 'Chrome/113.0.5672.94'})
    url_page = urlopen(url_link).read()
    url_html = bs(url_page, 'html.parser')
    
    containers = url_html.find_all('div', class_='content_episode revent datagrild')
    print("\rProcessing {}..".format(alphabet), end='')
    
    for items in containers[:]:
        animes = items.find_all('div',class_ = "name")
        html_links = items.find_all('a') #getting url of anime
        
        for i,anime in enumerate(animes[:]):
            name=''
            description=''
            season=''
            genre=''
            new_url=''
            status=''
            episodes=''
            status
            initial_air_date=''
            movie=''
            sub=''
            
            
            try:
                title = anime.text#find('div',class_ = "name").text #getting name of anime
#                 print(title)
                html_link = html_links[i].get('href') #getting url of anime
#                 print(main_url)
#                 print(base_url + html_link)

                #Checking if Season in mentioned in title & extracting both name and season seperatly
                #using  regex search function to extract Anime Title and Season seperatly
                if re.search(r"Season [0-9]?",title):
                    name,season = re.split(r"Season [0-9]?",title) #spliting title content based on " Season " text
                elif re.search(r"[0-9][a-z]+? Season",title):
                    name,season = re.split(r"[0-9][a-z]+? Season",title) #spliting title content based on " Season " text
                elif re.search(r"[0-9][a-z][a-z]",title):
                    try:
                        name,season = re.split(r"[0-9]['nd','rd','th']+",title) #spliting title content based on " Season " text
                    except:
                        name = re.split(r"[0-9]['nd','rd','th']+",title)[0] #spliting title content based on " Season " text
                else:
                    name = title #if no " Season " text is found in the Anime title Initilizing name = title

                #initilizing a pattern to search for " Season " in the Anime Title using regex
                pattern = r"^([0-9][a-z]+? Season | Season [0-9][a-z]+?\s )$"
                season = re.search(pattern,title) #using regex search function to search for pattern

                if not season:
                    pattern = r"[0-9]['nd','rd','th']+"
                    season = re.search(pattern,title) #using regex search function to search for pattern

                if season:
                    pass #if season is already found do nothing
                else:
                    season = ["Season 1"] #if season is still not found Initilizing season = ["Season 1"]
                
                #Initilizing " sub " variable as 'Dub' or 'Sub' based on its Apperance in Title
                if 'Dub' in title or 'dub' in title:
                    sub = 'Dub'
                else:
                    sub = 'Sub'
                

                #creating new_url to reroute to anime main page to extract more detatiled information such as:
                #1. description
                #2. status
                #3. initial air date
                #4. genre
                #5. total number of episodes
                new_url = base_url+html_link
                new_url = new_url.replace('watch','anime')
#                 print(main_url)
#                 print(html_link)
                url_link = Request(new_url, headers={'User-Agent': 'Chrome/113.0.5672.94'})
                url_page = urlopen(url_link).read()
                url_html = bs(url_page, 'html.parser')

                #getting div's with class_="main_body"
                containers = url_html.find_all('div',class_="main_body")
                #getting div's with class_="right"
                contents = containers[0].find_all('div',class_='right')


                #extracting description of anime
                description = contents[0].find_all('p')[0].text 
                #extracting status, initial air date, genre & total number of episodes of anime
                temp=[]
                genre=[]
                for i in contents[0].find_all('a')[2:]:
                    temp.append(i.text)

                status,initial_air_date = temp[:2]
                genre = temp[2:]
                genre = ', '.join(genre)

                containers = containers[0].find_all('div',class_='list_episode')
                try:
                    text = containers[0].find('span',class_='name').text
                    text = re.search('Episode [0-9]+',text)
                    episodes = re.search('[0-9]+',text[0])[0]
                    if int(episodes) > 0 and int(episodes) < 2:
                        movie = 'Movie'
                    else:
                        movie = 'Series'
                except:
                    episodes = 'None'
                    movie = 'None'


                print("\rProcessing {}...".format(alphabet),end='')

                #adding all extracted information in Anime dict
                anime_dict['Anime Title'] = name
                anime_dict['Description'] = description
                anime_dict['Current/Latest Season'] = season[0]
                anime_dict['URL'] = new_url
                anime_dict['Status'] = status
                anime_dict['Initial Air Date'] = initial_air_date
                anime_dict['Genre'] = genre
                anime_dict['Episodes Aired'] = episodes
                anime_dict['Series/Movie'] = movie
                anime_dict['Sub/Dub'] = sub

                new_dict = anime_dict.copy()

                Anime.append(new_dict)
                
            except:

                print("\rProcessing {}...".format(alphabet),end='')
                
                try:
                    #adding all extracted information in Anime dict
                    anime_dict['Anime Title'] = name
                    anime_dict['Description'] = description
                    anime_dict['Current/Latest Season'] = season[0]
                    anime_dict['URL'] = new_url
                    anime_dict['Status'] = status
                    anime_dict['Initial Air Date'] = initial_air_date
                    anime_dict['Genre'] = genre
                    anime_dict['Episodes Aired'] = episodes
                    anime_dict['Series/Movie'] = movie
                    anime_dict['Sub/Dub'] = sub
                except:
                    pass

                new_dict = anime_dict.copy()

                Anime.append(new_dict)
                
#                 break
    
    print("\rProcessed {}     ".format(alphabet))
#     break
    
main_end = time.time() - main_start
print('\rWeb Scraping Complete')
print("Total Time Taken: {}s".format(main_end))

Processed X     
Processed Y     
Processed Z     
Web Scraping Complete
Total Time Taken: 11897.706387043s


In [14]:
anime_dict

{'Anime Title': 'Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai - MOVIE',
 'Description': 'Kokuhaku Jikkou Iinkai: Renai Series (music and animation series featuring seiyuus) created by HoneyWorks getting an animated movie to premiere Spring 2016.\r\n\r\n',
 'Current/Latest Season': 'Season 1',
 'URL': 'https://animeseries.so//anime/zutto-mae-kara-suki-deshita-kokuhaku-jikkou-iinkai-movie.html',
 'Status': 'Completed',
 'Initial Air Date': '2016',
 'Genre': 'Romance, School',
 'Episodes Aired': '1',
 'Series/Movie': 'Movie',
 'Sub/Dub': 'Sub'}

In [15]:
print(len(Anime))

12245


In [16]:
Anime[0]

{'Anime Title': '.Hack//G.U. Returner',
 'Description': 'hack//G.U. Returner is a single-episode OVA offered to fans who completed all three GU Games, featuring characters from the .hack//G.U. Games and .hack//Roots.',
 'Current/Latest Season': 'Season 1',
 'URL': 'https://animeseries.so//anime/hackgu-returner.html',
 'Status': 'Completed',
 'Initial Air Date': '2007',
 'Genre': 'Adventure, Drama, Game, Harem, Martial Arts, Seinen',
 'Episodes Aired': '1',
 'Series/Movie': 'Movie',
 'Sub/Dub': 'Sub'}

In [17]:
print(len(Anime))

12245


In [11]:
print(Anime)

[{'Anime Title': '.Hack//G.U. Returner', 'Description': 'hack//G.U. Returner is a single-episode OVA offered to fans who completed all three GU Games, featuring characters from the .hack//G.U. Games and .hack//Roots.', 'Current/Latest Season': 'Season 1', 'URL': 'https://animeseries.so//anime/hackgu-returner.html', 'Status': 'Completed', 'Initial Air Date': '2007', 'Genre': 'Adventure, Drama, Game, Harem, Martial Arts, Seinen', 'Episodes Aired': '1', 'Series/Movie': 'Movie', 'Sub/Dub': 'Sub'}, {'Anime Title': '.hack//G.U. Trilogy', 'Description': 'Based on the CyberConnect2 HIT GAME, now will be released in a CG Movie!\r\n\r\nThe Movie will be placed in the storyline of each .hack//G.U. games trilogy. The story follows Haseo, a player in the online MMORPG called The World:R2 at first depicted as a PKK (Player Killer Killer) known as the "Terror of Death", a former member of the disbanded Twilight Brigade guild. Haseo encounters Azure Kite (believing him to be Tri-Edge and blaming him f

In [18]:
header = ['Anime Title', 'Description', 'Current/Latest Season', 'Episodes Aired', 'Status', 'Initial Air Date', 'Genre', 'Sub/Dub', 'Series/Movie', 'URL']

with open(r'E:\ML PROJECT\web_scraper\AnimeDB.csv', 'w+', newline='', encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=header)

    writer.writeheader()
    writer.writerows(Anime)
    
print("CSV file saved to: ",r'E:\ML PROJECT\web_scraper\AnimeDB.csv')

CSV file saved to:  E:\ML PROJECT\web_scraper\AnimeDB.csv


In [19]:
df = pd.read_csv("AnimeDB.csv")

In [20]:
df.head()

,Anime Title,Description,Current/Latest Season,Episodes Aired,Status,Initial Air Date,Genre,Sub/Dub,Series/Movie,URL
0,.Hack//G.U. Returner,hack//G.U. Returner is a single-episode OVA of...,Season 1,1,Completed,2007,"Adventure, Drama, Game, Harem, Martial Arts, S...",Sub,Movie,https://animeseries.so//anime/hackgu-returner....
1,.hack//G.U. Trilogy,"Based on the CyberConnect2 HIT GAME, now will ...",Season 1,1,Completed,2008,"Action, Game, Harem, Seinen",Sub,Movie,https://animeseries.so//anime/hackgu-trilogy.html
2,.hack//Gift,As an expression of gratitude for the heroes o...,Season 1,1,Completed,2003,"Comedy, Fantasy, Game",Sub,Movie,https://animeseries.so//anime/hackgift.html
3,.hack//Legend of the Twilight,.hack//Legend of the Twilight,Season 1,12,Completed,2003,"Adventure, Comedy, Fantasy, Sci-Fi, Shounen",Sub,Series,https://animeseries.so//anime/hacklegend-of-th...
4,.hack//Liminality,"The story follows three girls, Mai Minase, Yuk...",Season 1,4,Completed,2002,"Mystery, Sci-Fi",Sub,Series,https://animeseries.so//anime/hackliminality.html


In [26]:
import numpy as np

In [33]:
df[df.isnull().any(axis=1)]

,Anime Title,Description,Current/Latest Season,Episodes Aired,Status,Initial Air Date,Genre,Sub/Dub,Series/Movie,URL
26,"100,000 Levels of Body Refining : All the dogs...",NaN,"0,",87,Ongoing,2021,"Action, Comedy, Harem, Martial Arts, Shounen",Sub,Series,https://animeseries.so//anime/100000-levels-of...
31,11eyes,NaN,Season 1,NaN,NaN,NaN,NaN,Sub,NaN,https://animeseries.so//anime/11eyes-.html
93,A Favorite Marriage is Coming,NaN,Season 1,36,Completed,2019,Romance,Sub,Series,https://animeseries.so//anime/a-favorite-marri...
103,A Useless Onmyoji Family,NaN,Season 1,12,Completed,2019,"Comedy, Fantasy, Slice of Life",Sub,Series,https://animeseries.so//anime/a-useless-onmyoj...
166,Adversities,NaN,Season 1,1,Completed,2023,"Action, Adventure, Romance",Sub,Movie,https://animeseries.so//anime/adversities.html
...,...,...,...,...,...,...,...,...,...,...
12215,Zoids Wild,NaN,Season 1,7,Completed,2017,"Action, Adventure, Comedy, Mecha, Sci-Fi",Sub,Series,https://animeseries.so//anime/zoids-wild.html
12216,Zoids Wild (Dub),NaN,Season 1,25,Completed,2018,"Action, Adventure, Comedy, Mecha, Sci-Fi",Dub,Series,https://animeseries.so//anime/zoids-wild-dub.html
12223,Zoku Owarimonogatari,NaN,Season 1,6,Completed,2018,"Comedy, Mystery, Supernatural, Vampire",Sub,Series,https://animeseries.so//anime/zoku-owarimonoga...
12232,Zombieland Saga Movie,NaN,Season 1,None,Upcoming,0,"Comedy, Supernatural",Sub,None,https://animeseries.so//anime/zombieland-saga-...
